In [1]:
import numpy as np
from numpy import *
from random import shuffle
import math
import pandas as pd
from sklearn.metrics import r2_score

In [2]:
def data_generate(M,P_PDG,W,N,K,Noise):
    #W是原数据集,代表哪些代理之间有连接，即网络
    #N是W的行数，即代理的个数
    #代理人的博弈策略
    player = zeros((M+1,N))
    #原始博弈策略放在第一行，随机设置
    for i in range(N):
        if random.random()<=0.5:
            player[0,i] = 1 #1代表合作
        else:
            player[0,i] = 0 #0代表不合作
    #计算每个节点的收益
    #M轮博弈
    F = zeros((1,N))
    G = zeros((M,N))
    A = zeros((N,M,N))
    for t in range(M):
        for i in range(N):
            if player[t,i] == 1:
                s1 = array([[1],[0]])
            else:
                s1 = array([[0],[1]])
            for j in range(N):
                if player[t,j] == 0:
                    s2 = array([[0],[1]])
                else:
                    s2 = array([[1],[0]])
                F[0,j] = ((s1.T).dot(P_PDG)).dot(s2) #如果代理i和j连接，则代理i的收益为F[0,j]
            A[i,t,:] = F  #A:N*M*N   
            # F是三维矩阵A的第i页，第t行
            G[t,i] = F.dot(W[:,i]) #第t轮代理i的收益
        # update strategies
        for k in range(N):
            s=[i for i,x in enumerate(list(W[:,k])) if x>=1]     # 找出与代理k合作的代理的索引
            if len(s)!=0: # 如果有代理与代理k合作
                shuffle(s)
                P = 1/(1+math.e**((G[t,k]-G[t,s[0]])/K)) # 费米规则
                if random.random()<= P:
                    player[t+1,k] = player[t,s[0]]
                else:
                    player[t+1,k] = player[t,k]
            else: # 如果没有代理与代理k合作
                player[t+1,k] = player[t,k]
    # add noise for G
    Aa = G + Noise*random.random((M,N)) # m轮收益总矩阵：包含每一轮的收益M*N
    return [A,Aa]

In [3]:
def TrainSTRidge(R, Ut, lam, d_tol, maxit = 25, STR_iters = 10, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge.

    It runs over different values of tolerance and trains predictors on a training set, then evaluates them 
    using a loss function on a holdout set.

    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]       

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.001*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = np.linalg.lstsq(TrainR, TrainY)[0]
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge(R,Ut,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: 
        print ("Optimal tolerance:", tol_best)
    return w_best

def STRidge(X0, y, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse 
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.

    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0
    
    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
    else: w = np.linalg.lstsq(X,y)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]
    
    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]
            
        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)
            
        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0: 
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds
        
        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]
    
    if normalize != 0: return np.multiply(Mreg,w)
    else: return w

In [7]:
M = 10 #m轮更新
b = 1.2 #叛逃者的收益
K = 0.1
P_PDG = array([[1,0],[b,0]]) #2*2的收益矩阵
Noise = 1
W = np.loadtxt("karate.txt") 
T = W.shape[0] #全部社区内个体的总数量
SV = 6
# the part for generate EG data
y = zeros((SV*M,T))
AA = zeros((T,SV*M,T))
for i in range(SV): #产生6组A和Y的值
    [A, Aa] = data_generate(M,P_PDG,W,T,K,Noise)
    for j in range(M):
        y[i*M+j,:] =Aa[j,:]
        for k in range(T):
            AA[k,i*M+j,:] = A[k,j,:]

In [8]:
X=zeros((T,T))
for j in range(T):
    w = TrainSTRidge(AA[j], y[:,j].reshape(-1,1),10**-1,1)
    for i in range(w.shape[0]):
        X[i,j]=w[i,0]
np.savetxt("X.txt", X,fmt='%f',delimiter=' ')

<ipython-input-3-13ac3e2d7f51>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-3-13ac3e2d7f51>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
<ipython-input-3-13ac3e2d7f51>:102: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advis

In [9]:
print(Noise*random.random((M,T)) )

[[3.24141008e-01 1.49674867e-01 2.22321388e-01 3.86488981e-01
  9.02598476e-01 4.49949990e-01 6.13063458e-01 9.02348583e-01
  9.92803504e-02 9.69809068e-01 6.53140036e-01 1.70909585e-01
  3.58152167e-01 7.50686141e-01 6.07830669e-01 3.25047229e-01
  3.84254265e-02 6.34274058e-01 9.58949269e-01 6.52790317e-01
  6.35058874e-01 9.95299568e-01 5.81850329e-01 4.14368588e-01
  4.74697502e-01 6.23510101e-01 3.38007615e-01 6.74752322e-01
  3.17201742e-01 7.78345482e-01 9.49571053e-01 6.62526867e-01
  1.35716356e-02 6.22846096e-01]
 [6.73659631e-01 9.71945002e-01 8.78193471e-01 5.09624377e-01
  5.57146937e-02 4.51159215e-01 1.99876654e-02 4.41710921e-01
  9.79586729e-01 3.59444464e-01 4.80893531e-01 6.88661183e-01
  8.80475889e-01 9.18235466e-01 2.16822138e-01 5.65188867e-01
  8.65102561e-01 5.08968961e-01 9.16722954e-01 9.21157610e-01
  8.31124926e-02 2.77718561e-01 9.35670486e-03 8.42342080e-01
  6.47174140e-01 8.41386119e-01 2.64730164e-01 3.97820753e-01
  5.52821480e-01 1.64940460e-01 3.698

In [10]:
def TrainSTRidge_SPL(R, Ut, v, lam, d_tol, maxit = 25, STR_iters = 10, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge_SPL.

    It runs over different values of tolerance and trains predictors on a training set, then evaluates them 
    using a loss function on a holdout set.

    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]       

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.001*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = np.linalg.lstsq(TrainR, TrainY)[0]
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge_SPL(R,Ut,v,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: 
        print ("Optimal tolerance:", tol_best)
    return w_best

def STRidge_SPL(X0, y, v, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse 
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.

    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0
    
    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
    else: w = np.linalg.lstsq(X*v,y*v)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]
    
    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]
            
        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)
            
        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0: 
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds
        
        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]*v) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y*v))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds]*v,y*v)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds]*v,y*v)[0]
    
    if normalize != 0: return np.multiply(Mreg,w)
    else: return w

In [28]:
X=zeros((T,T))
v = ones((SV*M,1))
v1 = zeros((SV*M,1))
Los = zeros((SV*M,T))
losss = zeros((SV*M,1))
lambda_0=0.3
t=0
while (t<13):  
    if (v==v1).all():
        break
    else:
        for j in range(T):
            w = TrainSTRidge_SPL(AA[j], y[:,j].reshape(-1,1), v,10**-1,1)
            Lo =  abs((np.dot(AA[j], w))-y[:,j].reshape(-1,1))
            for i in range(SV*M):
                Los[i,j]=Lo[i]
            for i in range(w.shape[0]):
                X[i,j]=w[i,0]
        for i in range(SV*M):
            losss[i]=np.mean(Los[i,:])
        t = t+1
        for i in range(SV*M):
            if losss[i]<lambda_0:
                v[i]=1-losss[i]/lambda_0
            else:
                v[i]=0
        lambda_0 = lambda_0*1.25
        #lambda_0 = lambda_0 + np.mean(losss)      
np.savetxt("X1.txt", X,fmt='%f',delimiter=' ')

<ipython-input-10-5e04d6e95ebc>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-10-5e04d6e95ebc>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
<ipython-input-10-5e04d6e95ebc>:102: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning w

In [29]:
v

array([[0.9208732 ],
       [0.92813227],
       [0.9265653 ],
       [0.93767183],
       [0.93002006],
       [0.92408387],
       [0.91734411],
       [0.93588241],
       [0.93056869],
       [0.93276318],
       [0.91655288],
       [0.92866714],
       [0.92562782],
       [0.93707048],
       [0.93091641],
       [0.91366097],
       [0.9336348 ],
       [0.9268876 ],
       [0.92965922],
       [0.93620847],
       [0.92261815],
       [0.91374636],
       [0.88144799],
       [0.89414322],
       [0.8894914 ],
       [0.88044605],
       [0.89239073],
       [0.86488646],
       [0.89992771],
       [0.88738778],
       [0.90038967],
       [0.89722606],
       [0.90149313],
       [0.89921199],
       [0.8768321 ],
       [0.88489861],
       [0.89509714],
       [0.88476705],
       [0.88965238],
       [0.90161287],
       [0.9341564 ],
       [0.94174511],
       [0.947561  ],
       [0.94707064],
       [0.93615883],
       [0.93002439],
       [0.92934234],
       [0.942

In [44]:
X=zeros((T,T))
v = ones((SV*M,T))
v1 = zeros((SV*M,T))
Los = zeros((SV*M,T))
lambda_0=0.3
t=0
while (t<13):
    if (v==v1).all():
        break
    else:
        for j in range(T):
            w = TrainSTRidge_SPL(AA[j], y[:,j].reshape(-1,1), v[:,j].reshape(-1,1),10**-1,1)
            Lo =  abs((np.dot(AA[j], w))-y[:,j].reshape(-1,1))
            for i in range(SV*M):
                Los[i,j]=Lo[i]
            for i in range(w.shape[0]):
                X[i,j]=w[i,0]
        t = t+1
        for j in range(T):
            for i in range(SV*M):
                if Los[i,j]<lambda_0:
                    v[i,j]=1-Los[i,j]/lambda_0
                else:
                    v[i,j]=0
        #lambda_0 = lambda_0*1.05
        lambda_0 = lambda_0 + np.mean(Los)
np.savetxt("X2.txt", X,fmt='%f',delimiter=' ')

<ipython-input-10-5e04d6e95ebc>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-10-5e04d6e95ebc>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
<ipython-input-10-5e04d6e95ebc>:102: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning w

In [45]:
v

array([[0.87105127, 0.85327571, 0.9561944 , ..., 0.97500396, 0.96679882,
        0.97161924],
       [0.95537657, 0.99194657, 0.99095378, ..., 0.96916508, 0.8972592 ,
        0.92429672],
       [0.94641828, 0.91390407, 0.99705801, ..., 0.99156318, 0.90389361,
        0.97105955],
       ...,
       [0.63535484, 0.9419794 , 0.97166635, ..., 0.93522197, 0.93033063,
        0.95050489],
       [0.89458023, 0.92616494, 0.98811372, ..., 0.93303618, 0.98417221,
        0.9852479 ],
       [0.94819044, 0.93486808, 0.93273317, ..., 0.87977302, 0.92362387,
        0.95843202]])